In [1]:
import torchvision.transforms as transforms 
import numpy as np 
import cv2 
import os 
from tqdm import tqdm
from deepface import DeepFace 

In [9]:
def get_photos(src_folder_path):
    photos_arr = []
    for filename in os.listdir(src_folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
            photos_arr.append(filename)
    return photos_arr


def transform_dataset(scr_directory, dest_directory, db_directory):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
    ])

    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    if not os.path.exists(db_directory):
        os.makedirs(db_directory)

    for foldname in tqdm(os.listdir(scr_directory), desc=f'Cheking diretory {scr_directory}'):
        src_folder_path = os.path.join(scr_directory, foldname)
        
        if os.path.isdir(src_folder_path):
            photos_arr = get_photos(src_folder_path)
            np.random.shuffle(photos_arr)
            for i, filename in enumerate(photos_arr):
                src_image_path = os.path.join(src_folder_path, filename)
                dest_image_path = os.path.join(dest_directory, foldname + filename)
                db_image_path = os.path.join(db_directory, foldname + filename)
                try:
                    face_obj = DeepFace.extract_faces(img_path=src_image_path, detector_backend='yolov8')
                    face_image = face_obj[0]['face']
                    rgb_face_image = (face_image * 255).astype(np.uint8)
                    bgr_face_image = cv2.cvtColor(rgb_face_image, cv2.COLOR_RGB2BGR)
                    bgr_face_image = (transform(bgr_face_image).permute(1, 2, 0).numpy() * 255).astype(np.uint8)
                    # if i < len(photos_arr) // 2:
                    if i < 1: # temporarry solution
                        cv2.imwrite(dest_image_path, bgr_face_image)
                    else:
                        cv2.imwrite(db_image_path, bgr_face_image)
                except Exception as e:
                    print(f'Error transforming data {src_image_path}: {e}')

src_directory_path = 'photos'
dest_directory_path = 'photos_transformed'
db_directory_path = 'photos_db'

transform_dataset(src_directory_path, dest_directory_path, db_directory_path)

Cheking diretory photos:   0%|          | 0/13 [00:00<?, ?it/s]

Cheking diretory photos: 100%|██████████| 13/13 [00:26<00:00,  2.03s/it]


In [ ]:
def check_prediction(path1, path2):
    filename1 = path1.split('.')[0][:-1]

    filename2 = path2.split('/')[-1]
    filename2 = filename2.split('.')[0][:-1]
    if filename1 == filename2:
        return True
    else:
        return False

db_path = 'photos_db'
scr_directory = 'photos_transformed'

for filename in tqdm(os.listdir(scr_directory), desc=f'Cheking images'):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
        src_image_path = os.path.join(scr_directory, filename)
        try:
            finding_res = DeepFace.find(src_image_path, db_path=db_path, silent=True, threshold=0.7)
            print(f'Trying to find: {src_image_path}')
            print('Found:')
            try:
                print(finding_res[0][['identity', 'distance']])
                if check_prediction(filename, finding_res[0]['identity'][0]):
                    print('✅')
                else:
                    print('❌ mistake')
            except Exception as e:
                print('❌ nothing found')

            print('-' * 40)
        except Exception as e:
            print(f'Exception occured: {e}')
            print('-' * 40)